# Assessment of the incident power density on realistic spherical human head model

In [ ]:
import os
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir(os.pardir)
import time

import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.special import roots_legendre

from src.field import poynting
from src.utils.dataloader import (load_antenna_el_properties,
                                  load_sphere_coords)
from src.utils.derive import holoborodko
from src.utils.integrate import elementwise_dblquad

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.rcParams.update({'text.usetex': True})
seaborn.set(style='whitegrid', context='paper', palette='colorblind', font='serif', font_scale=2)

In [ ]:
d = jnp.array([5, 10, 15, 25, 35, 45]) / -1000. # distance from the antenna
f = jnp.array([3.5, 6., 10., 26., 30., 60., 80., 100.])  # frequencies in GHz
p = 11  # number of root points

In [ ]:
def cart2sph(x, y, z):
    """Return spherical given Cartesain coordinates."""
    r = jnp.sqrt(x ** 2 + y ** 2 + z ** 2)
    theta = jnp.arccos(z / r)
    phi = jnp.arctan2(y, x)
    return r, theta, phi


def sph2cart(r, theta, phi):
    """Return Cartesian given Spherical coordinates."""
    x = r * jnp.cos(phi) * jnp.sin(theta)
    y = r * jnp.sin(phi) * jnp.sin(theta)
    z = r * jnp.cos(theta)
    return x, y, z


def simulate(f, d, p):
    """Return Cartesian given Spherical coordinates."""
    assert f in [3.5, 6., 10., 26., 30., 60., 80., 100.]
    if f < 30:
        target_area = (2 / 100, 2 / 100)  # effective irradiated surface area, 4cm2
    else:
        target_area = (1 / 100, 1 / 100)  # effective irradiated surface area, 1cm2
    A = target_area[0] * target_area[1]  # area of the effective irradiated surface
    
    # source data
    f = f * 1e9
    antenna_data = load_antenna_el_properties(f)
    Is = antenna_data.ireal.to_numpy() + antenna_data.iimag.to_numpy() * 1j
    Is = jnp.asarray(Is)
    xs = antenna_data.x.to_numpy()
    
    # antenna position -- coordinates for planar model
    xs = antenna_data.x.to_numpy()
    xs = jnp.asarray(xs)
    ys = jnp.zeros_like(xs)
    zs = jnp.zeros_like(xs)
    
    # target coordinates for planar model
    xt_planar = jnp.linspace(-target_area[0] / 2, target_area[0] / 2, p) + xs[-1] / 2
    yt_planar = jnp.linspace(-target_area[1] / 2, target_area[1] / 2, p)
    zt_planar = d
    
    # incident power density for planar model
    dx = xs[1] - xs[0]
    Is_x = holoborodko(Is, dx)
    S = np.empty((xt_planar.size, xt_planar.size), dtype=np.complex64)
    for xi, _xt in enumerate(xt_planar):
        for yi, _yt in enumerate(yt_planar):
            _Sx, _Sy, _Sz = poynting(_xt, _yt, zt_planar, xs, ys, zs, f, Is, Is_x)
            _S = jnp.sqrt(_Sx ** 2 + _Sy ** 2 + _Sz ** 2)
            S[xi, yi] = _S
    Sab_planar = 1 / (2 * A) * elementwise_dblquad(S.real, xt_planar, yt_planar, p)
       
    # target coordinates for curved model
    data = load_sphere_coords(2312)  # spherical head model coordinates
    target_area_origin = (-target_area[0]/2, -target_area[1]/2)
    data_target = data[  # effective irradiated skin surface
        (data['y'] < 0) &
        (data['x'] > target_area_origin[0]) &
        (data['x'] < target_area_origin[0] * -1) &
        (data['z'] > target_area_origin[1]) &
        (data['z'] < target_area_origin[1] * -1)]
    data_target.reset_index(drop=True, inplace=True)
    xt_spherical = data_target['x'].to_numpy()
    yt_spherical = data_target['y'].to_numpy()
    zt_spherical = data_target['z'].to_numpy()
    
    # antenna position -- coordinates for curved model
    xs = xs - xs.max() / 2
    ys = jnp.zeros_like(xs) + data_target['y'].min() + d
    zs = jnp.zeros_like(xs)
    dx = xs[1] - xs[0]
    Is_x = holoborodko(Is, dx)
    
    # generate Gaussian points in spherical c.s. and convert to Cartesian c.s.
    _, theta, phi = cart2sph(xt_spherical, yt_spherical, zt_spherical)
    theta_a = theta.min()
    theta_b = theta.max()
    phi_a = phi.min()
    phi_b = phi.max()
    theta_points, theta_weights = roots_legendre(p)
    phi_points, phi_weights = roots_legendre(p)
    theta_points = 0.5 * (theta_points + 1.) * (theta_b - theta_a) + theta_a
    theta_weights = 0.5 * theta_weights * (theta_b - theta_a)
    phi_points = 0.5 * (phi_points + 1.) * (phi_b - phi_a) + phi_a
    phi_weights = 0.5 * phi_weights * (phi_b - phi_a)
    r = 0.09
    
    # incident power density for curved model
    poynting_mag = 0
    A = 0
    for _theta, _w_theta in zip(theta_points, theta_weights):
        for _phi, _w_phi in zip(phi_points, phi_weights):
            # normal vector components
            nx = r ** 2 * jnp.cos(_phi) * jnp.sin(_theta) ** 2 
            ny = r ** 2 * jnp.sin(_phi) * jnp.sin(_theta) ** 2
            nz = r ** 2 * jnp.cos(_theta) * jnp.sin(_theta)
            # power density vector field
            _xt, _yt, _zt = sph2cart(r, _theta, _phi)
            _Sx, _Sy, _Sz = poynting(_xt, _yt, _zt, xs, ys, zs, f, Is, Is_x)
            # dot product between power density and normal vector
            Fn = _Sx.real * nx + _Sy.real * ny + _Sz.real * nz
            # Gaussian quadrature
            poynting_mag += Fn * _w_theta * _w_phi
            A += np.sin(_theta) * r ** 2 * _w_theta * _w_phi
    Sab_spherical = poynting_mag / (2 * A)
    return Sab_planar.item(), Sab_spherical.item()

In [ ]:
res = np.zeros((d.size * f.size, 5))
i = 0
for _d in d:
    for _f in f:
        Sab_planar, Sab_spherical = simulate(_f, _d, p)
        res[i, :] = [_f, _d, p, Sab_planar, Sab_spherical]
        i += 1

In [ ]:
fig = plt.figure(figsize=(13, 5))
n_rows = 2
n_cols = 3
axs = fig.subplots(n_rows, n_cols, sharex=True)
start_idx = 0
end_idx = f.size
d_idx = 0
for row in range(n_rows):
    for col in range(n_cols):
        axs[row, col].plot(f, res[start_idx:end_idx, 3], 'b-', lw=3, label='planar')
        axs[row, col].plot(f, res[start_idx:end_idx, 4], 'r-', lw=3, label='spherical')
        if col == 0:
            axs[row, col].set(ylabel='IPD [W/m$^2$]')
        if row == 1:
            axs[row, col].set(xlabel='f [GHz]')
        axs[row, col].set(title=f'd = ${np.abs(d[d_idx]) * 1e3:.2f}$ mm')
        start_idx = end_idx
        end_idx += f.size
        d_idx += 1
handles, labels = plt.gca().get_legend_handles_labels()
fig.legend(handles, labels, loc='center right')
fig.tight_layout()
fig.subplots_adjust(right=0.825);